In [502]:
from Funciones import*


In [503]:
from keras.models import load_model
import numpy as np

import serial

flagADC = True

# Configura el puerto serial, ajusta el nombre del puerto según sea necesario
puerto_serial = serial.Serial(
    "COM4", 115200
)  # Reemplaza 'COM3' con el nombre de tu puerto


def read_binary_data(serial_port, data_size):
    data = serial_port.read(data_size)  # Lee los datos binarios
    return int.from_bytes(data, byteorder="little")  # Convierte a un entero


data_size = 2
end_of_transmission_char = 0x04

matriz_datos = []


datos_fila = []

    # Escuchar continuamente al puerto serial
while flagADC:
        # Intentar leer un dato
    try:
        linea_serial = read_binary_data(puerto_serial, data_size)
        if(linea_serial<7000):
            datos_fila.append(float(linea_serial))
    except Exception as e:
        print(f"Error al leer datos: {e}")
        break

        # Verificar si se ha recibido el carácter especial
    if puerto_serial.in_waiting > 0:
        if ord(puerto_serial.read()) == end_of_transmission_char:
            flagADC= False
            break


puerto_serial.close()
print(datos_fila)


muestras_test=[]

muestras_test = datos_fila[:-15]

matriz_np = np.array(muestras_test, dtype=float)

muestras_test = matriz_np.reshape(1, -1)



coefs_test = []

wavelet = 'coif6' 
nivel_descomposicion = 3  

for muestras_rango in muestras_test:
    coeficientes = transformadaWavelet(muestras_rango,wavelet,nivel_descomposicion)
    coefs_test.append(coeficientes)


recons_test = []

for coefs in coefs_test:
    senal_recons = reconstruccionWavelet(coefs,wavelet)
    recons_test.append(senal_recons)

datos_analizar = [recons_test]

caract_MAV = calcular_caracteristica_lista(datos_analizar, calcular_MAV)
caract_STD = calcular_caracteristica_lista(datos_analizar, calcular_STD)
caract_RMS = calcular_caracteristica_lista(datos_analizar, calcular_RMS)
caract_VAR = calcular_caracteristica_lista(datos_analizar, calcular_VAR)
caract_WL = calcular_caracteristica_lista(datos_analizar, calcular_WL)
caract_DMAV = calcular_caracteristica_lista(datos_analizar, calcular_DMAV)
caract_SSC = calcular_caracteristica_lista(datos_analizar, calcular_SSC)

caract_Entropy = calcular_caracteristica_lista(datos_analizar, calcular_Entropy)
caract_Mean_Derivative = calcular_caracteristica_lista(datos_analizar, calcular_Mean_Derivative)

caract_numberlow = calcular_caracteristica_lista(datos_analizar, contar_valores_menores_200)


X = [caract_MAV,caract_STD,caract_RMS,caract_VAR,caract_WL,caract_DMAV,caract_SSC,caract_Entropy,caract_Mean_Derivative]

X = np.array(X).reshape(1, -1)



if caract_numberlow[0][0]<250:

    # Cargar el modelo desde el archivo
    loaded_model = load_model('NN Compiladas/modeloV2_V19.h5')

    # Realizar predicciones en datos de prueba
    predictions = loaded_model.predict(X)

    print(predictions)

    # Obtener las etiquetas predichas como enteros
    predicted_labels = np.argmax(predictions, axis=1)

    # Mapear los enteros a los nombres de dedos
    finger_mapping = {0: 'Dedo_Anular', 1: 'Dedo_Índice', 2: 'Dedo_Medio'}

    # Convertir los enteros a nombres de dedos
    predicted_fingers = [finger_mapping[label] for label in predicted_labels]

    # Imprimir los dedos predichos
    print("Dedos predichos:")
    print(predicted_fingers)
else:
    print("Ruido")


[395.0, 164.0, 262.0, 235.0, 335.0, 907.0, 139.0, 555.0, 1046.0, 626.0, 554.0, 265.0, 657.0, 615.0, 451.0, 642.0, 293.0, 1486.0, 585.0, 959.0, 541.0, 943.0, 380.0, 1270.0, 592.0, 1007.0, 272.0, 942.0, 1300.0, 1192.0, 415.0, 631.0, 1848.0, 814.0, 617.0, 1395.0, 1406.0, 1182.0, 389.0, 2101.0, 496.0, 724.0, 1264.0, 1411.0, 1559.0, 220.0, 1042.0, 1813.0, 745.0, 2787.0, 450.0, 1807.0, 1660.0, 2148.0, 340.0, 2054.0, 1117.0, 1483.0, 894.0, 969.0, 528.0, 658.0, 1648.0, 1726.0, 1866.0, 567.0, 1578.0, 1093.0, 1230.0, 1680.0, 588.0, 1463.0, 638.0, 1653.0, 495.0, 1133.0, 887.0, 972.0, 1031.0, 1231.0, 493.0, 1057.0, 1223.0, 596.0, 1099.0, 1441.0, 503.0, 868.0, 971.0, 2026.0, 664.0, 954.0, 661.0, 1108.0, 1378.0, 401.0, 1748.0, 895.0, 1424.0, 879.0, 1346.0, 738.0, 1736.0, 436.0, 1317.0, 396.0, 715.0, 1271.0, 1710.0, 569.0, 1716.0, 457.0, 1302.0, 979.0, 881.0, 702.0, 511.0, 1424.0, 487.0, 995.0, 502.0, 932.0, 1011.0, 1109.0, 918.0, 516.0, 706.0, 997.0, 1103.0, 710.0, 401.0, 1899.0, 253.0, 612.0, 991.0